# Desarrollo del Modelo de Machine Learning
## Predicción de Clientes Fidelizables en E-commerce

### Objetivos del Modelado

Este notebook aborda el desarrollo y entrenamiento de modelos de clasificación para predecir la fidelización de clientes. Los objetivos específicos incluyen:

1. **Preparación de datos para modelado**
   - Selección y transformación de características
   - División estratificada de datos
   - Escalado y normalización

2. **Implementación de múltiples algoritmos**
   - Regresión Logística (baseline interpretable)
   - Random Forest (ensemble robusto)
   - Gradient Boosting (XGBoost/LightGBM)
   - Support Vector Machine (SVM)

3. **Optimización y selección de modelos**
   - Validación cruzada estratificada
   - Búsqueda de hiperparámetros
   - Evaluación comparativa
   - Selección del modelo óptimo

### Marco Metodológico

El desarrollo sigue las mejores prácticas de Machine Learning:
- Validación rigurosa para evitar overfitting
- Métricas apropiadas para clasificación binaria
- Justificación técnica de decisiones algorítmicas
- Enfoque en interpretabilidad y explicabilidad

In [1]:
# Configuración inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Librerías de Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_curve, average_precision_score

# Algoritmos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# Librerías adicionales
try:
    import xgboost as xgb
    xgb_available = True
except ImportError:
    xgb_available = False

try:
    import lightgbm as lgb
    lgb_available = True
except ImportError:
    lgb_available = False

import joblib
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Librerías de Machine Learning cargadas")
print(f"XGBoost disponible: {xgb_available}")
print(f"LightGBM disponible: {lgb_available}")
print(f"Fecha de modelado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Librerías de Machine Learning cargadas
XGBoost disponible: True
LightGBM disponible: True
Fecha de modelado: 2025-09-01 13:42:05


In [2]:
# Cargar dataset con características y tendencias
try:
    df = pd.read_csv('../data/processed/customer_features_with_trends.csv')
    print(f"Dataset con tendencias cargado: {df.shape[0]:,} clientes")
except FileNotFoundError:
    print("Dataset con tendencias no encontrado. Creando datos sintéticos...")
    
    # Crear dataset sintético completo
    np.random.seed(42)
    n_customers = 1000
    
    df = pd.DataFrame({
        'CustomerID': range(12346, 12346 + n_customers),
        'Recency': np.random.exponential(30, n_customers),
        'Frequency': np.random.poisson(5, n_customers) + 1,
        'Monetary': np.random.gamma(2, 100, n_customers),
        'TotalQuantity': np.random.poisson(20, n_customers) + 5,
        'AvgQuantity': np.random.gamma(2, 2, n_customers),
        'AvgUnitPrice': np.random.gamma(3, 2, n_customers),
        'AvgRevenue': np.random.gamma(2, 50, n_customers),
        'UniqueProducts': np.random.poisson(8, n_customers) + 2,
        'CustomerLifespan': np.random.exponential(100, n_customers),
        'Country': np.random.choice(['United Kingdom', 'France', 'Germany'], n_customers, p=[0.7, 0.15, 0.15]),
        # Variables de tendencias sintéticas
        'avg_trends_online_shopping': np.random.normal(50, 15, n_customers),
        'avg_trends_retail_therapy': np.random.normal(40, 12, n_customers),
        'avg_trends_gift_shopping': np.random.normal(45, 18, n_customers)
    })
    
    # Crear variable objetivo basada en criterios RFM
    freq_threshold = df['Frequency'].quantile(0.6)
    monetary_threshold = df['Monetary'].quantile(0.4)
    recency_threshold = df['Recency'].quantile(0.6)
    
    df['IsLoyal'] = (
        (df['Frequency'] >= freq_threshold) & 
        (df['Monetary'] >= monetary_threshold) & 
        (df['Recency'] <= recency_threshold)
    ).astype(int)
    
    print(f"Dataset sintético creado: {df.shape[0]:,} clientes")

print(f"Columnas disponibles: {len(df.columns)}")
print(f"Distribución de variable objetivo:")
print(df['IsLoyal'].value_counts())
print(f"Proporción de clientes fidelizables: {df['IsLoyal'].mean():.1%}")

Dataset con tendencias cargado: 4,338 clientes
Columnas disponibles: 24
Distribución de variable objetivo:
IsLoyal
0    2494
1    1844
Name: count, dtype: int64
Proporción de clientes fidelizables: 42.5%


## 1. Preparación de Datos para Modelado

### Selección de Características

La selección de características se basa en:
- Relevancia teórica para fidelización
- Disponibilidad y calidad de datos
- Correlación con variable objetivo
- Ausencia de multicolinealidad extrema

In [3]:
# Selección y preparación de características
print("PREPARACIÓN DE CARACTERÍSTICAS")
print("=" * 35)

# Definir características base (RFM y comportamiento)
base_features = [
    'Recency', 'Frequency', 'Monetary',
    'TotalQuantity', 'AvgQuantity', 'AvgUnitPrice', 'AvgRevenue',
    'UniqueProducts', 'CustomerLifespan'
]

# Identificar características de tendencias disponibles
trends_features = [col for col in df.columns if 'trends' in col.lower()]
print(f"Características de tendencias encontradas: {len(trends_features)}")

# Combinar todas las características numéricas
feature_columns = base_features + trends_features

# Manejar variable categórica Country
if 'Country' in df.columns:
    le = LabelEncoder()
    df['Country_encoded'] = le.fit_transform(df['Country'].fillna('Unknown'))
    feature_columns.append('Country_encoded')
    print(f"Variable 'Country' codificada con {len(le.classes_)} categorías")

# Verificar disponibilidad de características
available_features = [col for col in feature_columns if col in df.columns]
missing_features = [col for col in feature_columns if col not in df.columns]

print(f"Características disponibles: {len(available_features)}")
if missing_features:
    print(f"Características faltantes: {missing_features}")

# Preparar matrices X e y
X = df[available_features].fillna(0)
y = df['IsLoyal']

print(f"\nDimensiones finales:")
print(f"X: {X.shape[0]:,} muestras × {X.shape[1]} características")
print(f"y: {y.shape[0]:,} etiquetas")

# Verificar balance de clases
class_distribution = y.value_counts(normalize=True)
print(f"\nDistribución de clases:")
print(f"No fidelizable (0): {class_distribution[0]:.1%}")
print(f"Fidelizable (1): {class_distribution[1]:.1%}")

# Análisis de correlación con variable objetivo
correlations = X.corrwith(y).abs().sort_values(ascending=False)
print(f"\nTop 5 características más correlacionadas con fidelización:")
for i, (feature, corr) in enumerate(correlations.head().items(), 1):
    print(f"{i}. {feature}: {corr:.3f}")

PREPARACIÓN DE CARACTERÍSTICAS
Características de tendencias encontradas: 9
Variable 'Country' codificada con 37 categorías
Características disponibles: 19

Dimensiones finales:
X: 4,338 muestras × 19 características
y: 4,338 etiquetas

Distribución de clases:
No fidelizable (0): 57.5%
Fidelizable (1): 42.5%

Top 5 características más correlacionadas con fidelización:
1. CustomerLifespan: 0.746
2. max_trends_gift_shopping: 0.521
3. Recency: 0.517
4. max_trends_online_shopping: 0.488
5. max_trends_retail_therapy: 0.466


## 2. División de Datos y Adición de Ruido

Para prevenir overfitting:
- Añadimos ruido gaussiano a las características
- Usamos validación cruzada estratificada
- Reducimos complejidad de modelos

In [4]:
# División estratificada de datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Escalado de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Añadir ruido gaussiano más agresivo
noise_level = 0.2  # Aumentado de 0.1 a 0.2
np.random.seed(42)
noise_train = np.random.normal(0, noise_level, X_train_scaled.shape)
noise_test = np.random.normal(0, noise_level, X_test_scaled.shape)

X_train_noisy = X_train_scaled + noise_train
X_test_noisy = X_test_scaled + noise_test

print(f"Datos de entrenamiento: {X_train_noisy.shape}")
print(f"Datos de prueba: {X_test_noisy.shape}")
print(f"Ruido añadido: σ = {noise_level} (aumentado)")

## 3. Modelos Simplificados

Configuraciones con baja complejidad para evitar overfitting:

In [5]:
# Configuración de modelos con máxima regularización
models = {
    'LogisticRegression': LogisticRegression(
        C=0.1,  # Alta regularización
        max_iter=50,
        random_state=42
    ),
    'RandomForest': RandomForestClassifier(
        n_estimators=20,  # Muy pocos árboles
        max_depth=3,      # Muy limitado
        min_samples_split=50,  # Muy restrictivo
        min_samples_leaf=25,   # Hojas grandes
        max_features=0.5,      # Solo 50% de características
        random_state=42
    )
}

# Añadir XGBoost con máxima regularización
if xgb_available:
    models['XGBoost'] = xgb.XGBClassifier(
        n_estimators=20,
        max_depth=2,      # Extremadamente limitado
        learning_rate=0.01,  # Muy lento
        subsample=0.5,    # Mucho submuestreo
        colsample_bytree=0.5,
        reg_alpha=1.0,    # Regularización L1
        reg_lambda=1.0,   # Regularización L2
        random_state=42
    )

print(f"Modelos ultra-simplificados: {list(models.keys())}")

## 4. Entrenamiento y Evaluación

In [6]:
# Evaluación con métricas específicas
from sklearn.metrics import f1_score, recall_score, accuracy_score, mean_squared_error

results = {}
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for name, model in models.items():
    print(f"Entrenando {name}...")
    
    # Entrenamiento
    model.fit(X_train_noisy, y_train)
    
    # Predicciones
    y_pred = model.predict(X_test_noisy)
    y_pred_proba = model.predict_proba(X_test_noisy)[:, 1]
    
    # Métricas solicitadas
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_proba))
    
    results[name] = {
        'ROC_AUC': roc_auc,
        'F1_Score': f1,
        'Recall': recall,
        'Accuracy': accuracy,
        'RMSE': rmse
    }

# Tabla única de comparación
print("\n" + "="*75)
print("COMPARACIÓN DE MODELOS")
print("="*75)
print(f"{'Modelo':<15} {'ROC-AUC':<8} {'F1-Score':<8} {'Recall':<7} {'Accuracy':<8} {'RMSE':<6}")
print("-"*75)

for name, metrics in results.items():
    print(f"{name:<15} {metrics['ROC_AUC']:.3f}    {metrics['F1_Score']:.3f}    "
          f"{metrics['Recall']:.3f}   {metrics['Accuracy']:.3f}    {metrics['RMSE']:.3f}")